In [88]:
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, ElementNotInteractableException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium import webdriver
import os
import re
import time
import datetime
import pickle
import warnings
import pandas as pd
from sys import platform
from tqdm import tqdm
warnings.filterwarnings('ignore')


chrome_opt = webdriver.ChromeOptions()


print('Current OS : ', platform)
if 'darwin' in platform:
    selen_path = '/Users/huni/Dropbox/내 Mac (MacBook-Pro.local)/Downloads/chromedriver'
    chrome_opt.add_argument('--kiosk')

elif 'win32' in platform:
    # to unable image
    chrome_opt.add_experimental_option("excludeSwitches", ["enable-logging"])
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_opt.add_experimental_option("prefs", prefs)
    chrome_opt.add_argument('--start-maximized')

    selen_path = r'C:\Users\jhun1\Dropbox\My PC (LAPTOP-VLNR6K8R)\Downloads\chromedriver_win32\chromedriver'



def setup(selenium=selen_path,
          option=chrome_opt):
    web_driver = webdriver.Chrome(
        executable_path=selen_path, chrome_options=chrome_opt)
    ACC = ActionChains(web_driver)
    return web_driver, ACC


def D1(main_page_lk,  # 메인페이지 링크
        test_opt=False,  # 테스트 할 때
       ):
    global driver, ACC
    driver.get(main_page_lk)
    df = []

    # crawler
    # 한줄 한줄 마우스 올려서 href 확인하기
    # 날짜, 제목, 링크, 발제자
    # Table = driver.find_element_by_class_name('notino-table-view')
    time.sleep(3)
    Table = driver.find_element_by_css_selector(
        '#notion-app > div > div.notion-cursor-listener > div > div.notion-frame > div.notion-scroller.vertical.horizontal > div:nth-child(3) > div > div > div.notion-selectable.notion-collection_view_page-block > div:nth-child(3)')
    if test_opt == True:
        print(type(Table))

    for idx in tqdm(range(1, 50), desc='>> crawling each page meta info..'):
        temp = {}

        try:
            each_block = Table.find_element_by_css_selector(
                f'#notion-app > div > div.notion-cursor-listener > div > div.notion-frame > div.notion-scroller.vertical.horizontal > div:nth-child(3) > div > div > div.notion-selectable.notion-collection_view_page-block > div:nth-child(3) > div:nth-child({idx+1})')
        except NoSuchElementException:
            print('crawling finished in: ', idx)
            break

        title_block = each_block.find_element_by_css_selector(
            'div:nth-child(5) > div:nth-child(1)')
        title = title_block.text
        temp['title'] = title
        day = each_block.find_element_by_css_selector('div:nth-child(1)').text
        temp['day'] = day
        people = each_block.find_element_by_css_selector(
            'div:nth-child(6)').text
        temp['people'] = people

        ACC.move_to_element(title_block).perform()
        # href_block = Table.find_element_by_css_selector(
        #     '#notion-app > div > div.notion-overlay-container.notion-default-overlay-container > div:nth-child(2) > div > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > a')
        open_block = title_block.find_element_by_css_selector('#notion-app > div > div.notion-overlay-container.notion-default-overlay-container > div:nth-child(2)')
        ACC.move_to_element(open_block).click().perform()
        link = open_block.get_attribute('href')

        # if test_opt == True and idx == 1:
        #     break
        # open_block =
        # ACC.move_to_element()
        # link = each_block.find_element_by_css_selector('pointer-events')

        temp['link'] = link
        df.append(temp)

        if test_opt == True:
            print(f'{idx}에서 {title}이 나오고, {people}이 발제했고, {day}에 이루어짐, {link}')

    return df

Current OS :  darwin


In [40]:
driver, ACC = setup()
rs1 = D1(main_page_lk='https://dndrl.notion.site/28b270dfd58d497cbd031f457f02ed92?v=c081bcadc88a486683c7e7325ea99174',
             test_opt=False,
             )

>> crawling each page meta info..:   0%|          | 0/49 [00:00<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#notion-app > div > div.notion-overlay-container.notion-default-overlay-container > div:nth-child(2)"}
  (Session info: chrome=98.0.4758.109)


In [72]:
Table = driver.find_element_by_css_selector(
        '#notion-app > div > div.notion-cursor-listener > div > div.notion-frame > div.notion-scroller.vertical.horizontal > div:nth-child(3) > div > div > div.notion-selectable.notion-collection_view_page-block > div:nth-child(3)')
each_block = Table.find_element_by_css_selector(
                f'#notion-app > div > div.notion-cursor-listener > div > div.notion-frame > div.notion-scroller.vertical.horizontal > div:nth-child(3) > div > div > div.notion-selectable.notion-collection_view_page-block > div:nth-child(3) > div:nth-child({2})')
title_block = each_block.find_element_by_css_selector(
            'div:nth-child(5) > div:nth-child(1)')

### try1 
# ACC.move_to_element_with_offset(title_block, 200, 0)
# ACC.click().perform()

### try2: https://stackoverflow.com/questions/12323403/how-do-i-find-an-element-that-contains-specific-text-in-selenium-webdriver-pyth
# test_block = title_block.find_element_by_xpath("//*[contains(text(), 'Theory')]")
# ACC.move_to_element(test_block)
# ACC.click() # perform까지가면 ElementNotInteractableException 뜸
# # t1.
# refind_table = driver.find_element_by_css_selector('#notion-app > div > div.notion-cursor-listener > div > div.notion-frame > div.notion-scroller.vertical.horizontal > div:nth-child(3) > div > div > div.notion-selectable.notion-collection_view_page-block > div:nth-child(3)')
# open_block = refind_table.find_element_by_xpath("//*[contains(text(), 'OPEN')]")
# ACC.move_to_element(open_block)
# ACC.click()
# ACC.perform() 


### try3: pyautogui
import pyautogui as pg
from PIL import ImageGrab
from functools import partial
ImageGrab.grab = partial(ImageGrab.grab, all_screens=True)
ACC.move_to_element(title_block)
try:
        ACC.perform() # to prevent ElementNotInteractableException
except ElementNotInteractableException:
        print('에러 발생:')
        pdf_con = pg.locateCenterOnScreen(f'{os.getcwd()}/OPEN.png', grayscale=True, confidence=0.7)
        print(pdf_con)
        pg.click()

에러 발생:
None


In [11]:
ACC.click(on_element=title_block).perform()
# 이러면 단순하게 제목 블록이 클릭만됨

In [9]:
open_block = title_block.find_element_by_css_selector('#notion-app > div > div.notion-overlay-container.notion-default-overlay-container > div:nth-child(2)')
ACC.move_to_element(open_block).click().perform()
link = open_block.get_attribute('href')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#notion-app > div > div.notion-overlay-container.notion-default-overlay-container > div:nth-child(2)"}
  (Session info: chrome=98.0.4758.109)


In [76]:
driver.get('https://dndrl.notion.site/1-Theory-and-Learning-Analytics-2f08bb791bc24e61b1167622774ebd8a')
time.sleep(3)
Content = driver.find_element_by_class_name('notion-page-content').text
Content


'학습분석을 활용하기 위해서는 우리는 알고리즘, 교수자, 학생, 정책가, 연구자 간 균형을 맞출 필요가 있다.\n빅데이터 시대에 이론의 중요성은 줄어들지 않는다.\n빅데이터를 통해 결과를 해석하고 활용하기 위해서는 이론은 중요하다.\n학습 분야도 다른분야에서 시작된 데이터 집약적(data-intensive) 방식으로 변화하고 있다.\n이를 위해 우리는 이론과 분석의 관계에서 "from clicks to constructs"라는 원칙을 이해할 필요가 있다.\n학습 분석은 이미 알고리즘을 일상적으로 사용하는 사회의 변화를 반영한다.\n따라서 이론과 학습 분석의 관계를 중심으로 하는 프레임은 일반적으로 "교육 이론"으로 간주되는 것을 고려할 수 없을 정도로 확장되어야 하며, 사회 기술 인프라가 사회에서 어떻게 계산 지능을 제공하는지에 대한 중요한 담론에 참여해야 한다.\n학습분석에서 고려해야 하는 것\n사람들이 감시 당한다는 것을 인지할 때 행동이 변화할 수 있기에, 학습 분석 연구자들은 "학습"이라는 렌즈가 특정 결과를 과장하거나 왜곡할 수 있다는 점을 인지해야 한다.\n학습분석을 통해, 교사와 학생들은 그동안 연구자들만이 확인할 수 있었던 교육과 학습 상황을 직접 확인할 수 있게 될 것이다.\n학습분석이 가져올 새로운 윤리적 딜레마를 고려할 필요가 있다.\n학습분석 분야는 인식론적 가정(epistemological assumptions)과 교육학적 실천(pedagogical practices)에 의해 평가 체제(assessment regimes)로 간주되어 왔다.\n그러나 학습 분석은 특정 학습자들을 길러내기 위해 설계된 교육적 관점(particular educaitonal worldview)이라 볼 수 있다.\n1. Theory into pratice\n삼각형의 사용은 각 요소들의 긴장을 강조한다. \n평가는 "지식"이 무엇인지 이해하는 데 있어 원동력이 될 수 있다. \n우리가 평가하는 사람과 방법에 대한 교육학에 대한 가정은 때때로 긴장 상태에 있으며, 종

In [78]:
driver.get('https://dndrl.notion.site/24-Addressing-the-Challenges-of-Institutional-Adoption-773c7ec0b37741aa820a275cf3af4cf1')
time.sleep(3)
Content_with_toggle = driver.find_element_by_class_name('notion-page-content').text
Content_with_toggle

'🤷🏻\u200d♀️\n요약 : "큰 규모의 학습 분석 연구를 실행하기 위해서 조직 내에서 \'현실적으로\' 필요한 부분에 대한 논의"\n1_REVIEW OF CURRENT MODELS OF LEARNING ANALYTICS DEPLOYMENT\n2_WHAT DO THE MODELS TELL US?\n3_BRINGING IN THE EMPIRICAL\n4_CONCLUSION\n참고문헌 \n🙋🏻\u200d♀️\n아래의 주제 중, 자유롭게 1~2개의 주제를 선택하여 논의해 보아요! \n✏️\n<현지, 지영, 준오 > \n2번 \n준오 : "학습 분석을 위한 적절한 학습 내용에 대한 고려" >> 여러 필드에 있는 사람들과 만나 교류 \nex) 학습 내용의 성격에 따라서 효과를 잘 드러내는 학습분석도 있을 것이고 (창의성/협업 등등)\n학습 분석 효과가 일관되지 않게 나타남 // "학습 내용과의 상호작용" > 비교 연구를 하기 어려움 > 여러 기술 현장\n1번\n지영 : Colvin et al.(2015) 기존의 연구와 크게 다를 것 없이 범위를 넓힌 것으로 보여 공감,  // 리더십 학습 분석으로 나쁜 결과를 얻었을 때 대응의 어려움 >> \n준오 : 복잡성도 학습 내용에 따라 다를텐데, 이것을 현실적으로 개별적으로 제시하기 어려움 \n현지 :  \n조직문화 > 그럴 수 있죠 // 일단 그대로 하고, 짬차면 살짝씩 🤣\n데이터 수집, 데이터 전처리, \n교육 전문가 (내적 타당도)\n도메인 전문가 \n똑똑 수학탐험대 → 국가수준에서 만든 서비스이지만, 현실적으로는 여기서 나온 데이터로도 단순히 부진학생 퍼센트 정도를 보는 정도로 그치게 되지 않을까... 이 문제 원인은 조직문화에 있는 것으로 보인다.\n<수진, 홍규, 웅기, 재용👍>\nColvin et al.(2015)의 한계를 보완하는 방법\n비즈니스 맥락에서 활용되는 전략을 포함하여 좀더 실제적으로 학습분석 실행에 도움이 되는 방식으로 변형\nex) 그로스해킹, 퍼포먼스마케팅 등: 질적인 인사이트 + 양적인 데

## Toggle should be clicked

In [ ]:
# 

In [110]:
driver.get('https://dndrl.notion.site/24-Addressing-the-Challenges-of-Institutional-Adoption-773c7ec0b37741aa820a275cf3af4cf1')
time.sleep(3)

Content = driver.find_element_by_class_name('notion-page-content')
### Try1
# toggles = Content.find_elements_by_class_name('triangle')
# print(len(toggles)) # 모든 toggle 보다 더 많은 element가 보이는 것 같음

# for toggle in tqdm(toggles, desc='>> toggle click'):
#     # print(toggle)
#     try:
#         toggle = Content.find_element_by_class_name('triangle')
#         ACC.move_to_element(toggle).perform()
#         ACC.click().perform()
#     except StaleElementReferenceException:
#         print('에러다..')
        # continue

### Try2
toggles = Content.find_elements_by_css_selector('polygon')
print(len(toggles)) # same results with Try1

for idx, toggle in tqdm(enumerate(toggles), desc='>> toggle click'):
    # toggles_duplicate = Content.find_elements_by_css_selector('polygon')
    for _ in range(5):
        ACC.move_to_element(toggles_duplicate[idx])
        try:
            ACC.click() # perform 파트에서 에러가 나네
            ACC.perform()
        except StaleElementReferenceException:
            print('Stale 에러, ', _)


15


>> toggle click: 1it [00:00,  6.79it/s]

Stale 에러,  0
Stale 에러,  1
Stale 에러,  2
Stale 에러,  3
Stale 에러,  4
Stale 에러,  0
Stale 에러,  1


>> toggle click: 2it [00:00,  6.27it/s]

Stale 에러,  2
Stale 에러,  3
Stale 에러,  4
Stale 에러,  0
Stale 에러,  1
Stale 에러,  2
Stale 에러,  3


...

Stale 에러,  0
Stale 에러,  1
Stale 에러,  2
Stale 에러,  3
Stale 에러,  4
Stale 에러,  0
Stale 에러,  1


>> toggle click: 14it [00:02,  6.57it/s]

Stale 에러,  2
Stale 에러,  3
Stale 에러,  4
Stale 에러,  0
Stale 에러,  1
Stale 에러,  2
Stale 에러,  3


>> toggle click: 15it [00:02,  6.16it/s]

Stale 에러,  4


## StaleElementReferenceException 관련
- 아무리 오랜 시간 wait를 줘도, 에러가 난다. DOM 구축 방식에 문제가 있는게 아닐까